In [1]:
import numpy as np
import os
from embeddings import SentenceEmbeddingDataset, WordEmbeddingManager, create_embedding_dataloader
import utils
import torch

/Users/johnhenryrudden/anaconda3/envs/shaky_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/johnhenryrudden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
TRAIN_PATH = 'data/raw_train.txt'
tokenized_sentences = utils.process_training_data(TRAIN_PATH)

In [3]:
WORD2VEC_MODEL_PATH = 'data/word2vec.model'
word2vec_manager = WordEmbeddingManager(WORD2VEC_MODEL_PATH)

Model loaded successfully from data/word2vec.model


In [4]:
dataloader = create_embedding_dataloader(tokenized_sentences, word2vec_manager, seq_length=10, batch_size=4)

In [6]:
batch = next(iter(dataloader))

['I', 'had', 'forgot', 'myself', ';', 'am', 'I', 'not', 'king', '?']
['GLOUCESTER', ':', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['HERMIONE', ':', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['Now', ',', 'had', 'I', 'not', 'the', 'dash', 'of', 'my', 'former']


In [7]:
batch[0]

tensor([[ 9.7490e-03,  3.4944e-01, -1.5521e+00, -4.8783e-01, -1.8674e+00,
         -2.0111e+00,  7.6004e-01,  1.7382e+00, -1.1583e+00, -1.8240e+00,
         -1.9179e-01, -1.1141e+00, -1.1188e-02,  5.4272e-01, -2.1304e+00,
          1.3131e+00, -3.2889e-01,  1.0444e+00, -1.5566e+00,  7.1348e-01,
          1.8329e-01,  1.0571e-01,  3.8896e-01,  1.4192e+00, -1.5341e+00,
         -1.6192e-01, -3.2304e-01, -1.2976e+00, -2.3000e+00, -6.0186e-01,
         -6.0075e-01, -8.9927e-01, -9.1504e-01, -4.6005e-01,  6.6732e-01,
          1.4319e+00,  7.8773e-01, -5.6260e-01, -7.3820e-01,  4.8822e-01,
          9.3350e-01, -7.1573e-01, -9.6818e-01, -2.8207e-01,  2.5232e+00,
          9.9717e-02,  6.7150e-01, -3.1055e-01,  1.3472e+00, -4.8279e-01],
        [ 2.8930e-01,  2.9470e-01, -4.6906e-01,  4.8581e-01, -8.6729e-01,
         -8.3763e-01,  1.3069e+00,  1.6623e+00, -8.8692e-01, -3.4346e-01,
         -4.9788e-01, -5.2447e-01, -5.3914e-01,  8.9144e-01, -1.4282e+00,
          2.6334e-01,  1.8575e-01,  1